In [41]:
import os
import pandas as pd
from math import radians, cos, sin, asin, sqrt

In [42]:
def haversine(lon1, lat1, lon2, lat2,stype):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    if stype == 'km':
        r = 6371 # Radius of earth in kilometers
    else:
        r = 3956 # Radius of earth in miles
    d=round(c * r,3)
    return d


In [43]:
InputFileName='C:/VKHCG/03-Hillman/00-RawData/GB_Postcode_Warehouse.csv'
OutputFileName='C:/VKHCG/03-Hillman/01-Retrieve/01-EDS/02-Python/Retrieve_Incoterm_Chain_GB_Warehouse.csv'

In [44]:
print('Loading :',InputFileName)
Warehouse=pd.read_csv(InputFileName,header=0,low_memory=False)
Warehouse.head()

Loading : C:/VKHCG/03-Hillman/00-RawData/GB_Postcode_Warehouse.csv


,id,postcode,latitude,longitude
0,2,AB10,57.13514,-2.11731
1,3,AB11,57.13875,-2.09089
2,4,AB12,57.10100,-2.11060
3,5,AB13,57.10801,-2.23776
4,6,AB14,57.10076,-2.27073


In [45]:
WarehouseGood=Warehouse[Warehouse.latitude != 0]
WarehouseGood.head()

,id,postcode,latitude,longitude
0,2,AB10,57.13514,-2.11731
1,3,AB11,57.13875,-2.09089
2,4,AB12,57.10100,-2.11060
3,5,AB13,57.10801,-2.23776
4,6,AB14,57.10076,-2.27073


In [47]:
WarehouseGood['Warehouse_Name']=WarehouseGood.apply(lambda row:
            'WH-' + row['postcode']
            ,axis=1)
WarehouseGood.drop('id', axis=1, inplace=True)
WarehouseGood.drop('postcode', axis=1, inplace=True)
WarehouseGood.head()

KeyError: ('postcode', 'occurred at index 0')

In [30]:
WarehouseFrom=WarehouseGood.head(100)
for i in range(WarehouseFrom.shape[1]):
    oldColumn=WarehouseFrom.columns[i]
    newColumn=oldColumn + '_from'
    WarehouseFrom.rename(columns={oldColumn: newColumn}, inplace=True)    
WarehouseFrom.insert(3,'Keys', 1)
WarehouseFrom.head()

,latitude_from,longitude_from,Warehouse_Name_from,Keys
0,57.13514,-2.11731,WH-AB10,1
1,57.13875,-2.09089,WH-AB11,1
2,57.10100,-2.11060,WH-AB12,1
3,57.10801,-2.23776,WH-AB13,1
4,57.10076,-2.27073,WH-AB14,1


In [48]:
WarehouseTo=WarehouseGood.head(100)
for i in range(WarehouseTo.shape[1]):
    oldColumn=WarehouseTo.columns[i]
    newColumn=oldColumn + '_to'
    WarehouseTo.rename(columns={oldColumn: newColumn}, inplace=True)    
WarehouseTo.insert(3,'Keys', 1)
WarehouseTo.head()

,latitude_to,longitude_to,Warehouse_Name_to,Keys
0,57.13514,-2.11731,WH-AB10,1
1,57.13875,-2.09089,WH-AB11,1
2,57.10100,-2.11060,WH-AB12,1
3,57.10801,-2.23776,WH-AB13,1
4,57.10076,-2.27073,WH-AB14,1


In [55]:
WarehouseCross=pd.merge(right=WarehouseFrom,
                       left=WarehouseTo,
                        how='outer',
                        on='Keys')
WarehouseCross.head()

,latitude_to,longitude_to,Warehouse_Name_to,Keys,latitude_from,longitude_from,Warehouse_Name_from
0,57.13514,-2.11731,WH-AB10,1,57.13514,-2.11731,WH-AB10
1,57.13514,-2.11731,WH-AB10,1,57.13875,-2.09089,WH-AB11
2,57.13514,-2.11731,WH-AB10,1,57.10100,-2.11060,WH-AB12
3,57.13514,-2.11731,WH-AB10,1,57.10801,-2.23776,WH-AB13
4,57.13514,-2.11731,WH-AB10,1,57.10076,-2.27073,WH-AB14


In [56]:
WarehouseCross.shape

(10000, 7)

In [57]:
WarehouseCross.drop('Keys', axis=1, inplace=True)
WarehouseCross.insert(0,'Incoterm', 'DDP')
WarehouseCross.head()

,Incoterm,latitude_to,longitude_to,Warehouse_Name_to,latitude_from,longitude_from,Warehouse_Name_from
0,DDP,57.13514,-2.11731,WH-AB10,57.13514,-2.11731,WH-AB10
1,DDP,57.13514,-2.11731,WH-AB10,57.13875,-2.09089,WH-AB11
2,DDP,57.13514,-2.11731,WH-AB10,57.10100,-2.11060,WH-AB12
3,DDP,57.13514,-2.11731,WH-AB10,57.10801,-2.23776,WH-AB13
4,DDP,57.13514,-2.11731,WH-AB10,57.10076,-2.27073,WH-AB14


In [59]:
WarehouseCross['DistanceBetweenKilometers'] = WarehouseCross.apply(lambda row: 
    haversine(
            row['longitude_from'],
            row['latitude_from'],
            row['longitude_to'],
            row['latitude_to'],
            'km')
            ,axis=1)
WarehouseCross.head()

,Incoterm,latitude_to,longitude_to,Warehouse_Name_to,latitude_from,longitude_from,Warehouse_Name_from,DistanceBetweenKilometers
0,DDP,57.13514,-2.11731,WH-AB10,57.13514,-2.11731,WH-AB10,0.000
1,DDP,57.13514,-2.11731,WH-AB10,57.13875,-2.09089,WH-AB11,1.644
2,DDP,57.13514,-2.11731,WH-AB10,57.10100,-2.11060,WH-AB12,3.818
3,DDP,57.13514,-2.11731,WH-AB10,57.10801,-2.23776,WH-AB13,7.872
4,DDP,57.13514,-2.11731,WH-AB10,57.10076,-2.27073,WH-AB14,10.020


In [60]:
WarehouseCross['DistanceBetweenMiles'] = WarehouseCross.apply(lambda row: 
    haversine(
            row['longitude_from'],
            row['latitude_from'],
            row['longitude_to'],
            row['latitude_to'],
            'miles')
            ,axis=1)
WarehouseCross.head()

,Incoterm,latitude_to,longitude_to,Warehouse_Name_to,latitude_from,longitude_from,Warehouse_Name_from,DistanceBetweenKilometers,DistanceBetweenMiles
0,DDP,57.13514,-2.11731,WH-AB10,57.13514,-2.11731,WH-AB10,0.000,0.000
1,DDP,57.13514,-2.11731,WH-AB10,57.13875,-2.09089,WH-AB11,1.644,1.021
2,DDP,57.13514,-2.11731,WH-AB10,57.10100,-2.11060,WH-AB12,3.818,2.371
3,DDP,57.13514,-2.11731,WH-AB10,57.10801,-2.23776,WH-AB13,7.872,4.888
4,DDP,57.13514,-2.11731,WH-AB10,57.10076,-2.27073,WH-AB14,10.020,6.222


In [ ]:
WarehouseCross.drop('longitude_from', axis=1, inplace=True) 
WarehouseCross.drop('latitude_from', axis=1, inplace=True) 
WarehouseCross.drop('longitude_to', axis=1, inplace=True) 
WarehouseCross.drop('latitude_to', axis=1, inplace=True)
WarehouseCross.head()

In [64]:
WarehouseCross.shape

(10000, 5)

In [66]:
WarehouseCrossClean=WarehouseCross[WarehouseCross.DistanceBetweenKilometers !=0]
WarehouseCrossClean.head()

,Incoterm,Warehouse_Name_to,Warehouse_Name_from,DistanceBetweenKilometers,DistanceBetweenMiles
1,DDP,WH-AB10,WH-AB11,1.644,1.021
2,DDP,WH-AB10,WH-AB12,3.818,2.371
3,DDP,WH-AB10,WH-AB13,7.872,4.888
4,DDP,WH-AB10,WH-AB14,10.020,6.222
5,DDP,WH-AB10,WH-AB15,2.919,1.813


In [68]:
WarehouseCrossClean.shape

(9900, 5)

In [69]:
print('###########')
print('Rows :',WarehouseCrossClean.shape[0])
print('Columns :',WarehouseCrossClean.shape[1])
print('###########')

###########
Rows : 9900
Columns : 5
###########


In [70]:
WarehouseCrossClean.to_csv(OutputFileName, index = False)

In [40]:
print('### Done!! ############################################')

### Done!! ############################################
